In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# NumPy beyond 2020

Ross Barnowski `rossbar@berkeley.edu` | [rossbar](https://github.com/rossbar) on GitHub

University of Michigan EECS | 1/30/2020

# What is NumPy?

> *NumPy is the fundamental package for scientific computing with Python*
> 
>  [numpy.org](https://numpy.org/)

Strong stuff.

In [ ]:
# Code example: github graphql query for top starred projects with numpy as a dependency

# A bit of history

 - **Mid 90's/Early 00's**: desire for high-performance numerical computation in python leads to [`numeric`](https://numpy.org/_downloads/768fa66c250a0335ad3a6a30fae48e34/numeric-manual.pdf)
 - Early adopters included the [Space Telescope Science Institute (STScI)](http://www.stsci.edu/) who developed another array computation package to better suit their needs: `numarray`.
 - **2005** The best ideas from `numeric` and `numarray` were combined in the development of a new library, `numpy`
   * This work was largely done by Travis Oliphant, then an assistant professor at BYU
 - **2006** Numpy v1.0 released
 
[NumPy Development History](https://github.com/numpy/numpy/graphs/contributors)

# What does NumPy provide?

 - `ndarray`: A generic, n-dimensional array data structure
 - Sophisticated machinery for operating on array data (broadcasting, `ufuncs`)
 - Tools for common scientific/numerical tasks:
   * Random number generation (`np.random`)
   * Fourier analysis (`np.fft`)
   * Linear algebra (`np.linalg`)
 - Language extension/integration (C-API, `f2py`)

# Where is NumPy used?

### <font color=red> Investigate the following for appropriately sized examples </font>

 - To produce the first image of a black hole 
   [Event Horizon Telescope Collaboration](https://github.com/achael/eht-imaging)
 - [To detect the gravitational wave signature from a neutron star merger](https://github.com/gwastro/pycbc)
 - [To discover fundamental particles like the Higgs Boson](https://github.com/cms-sw/cmssw)
   * Also [scikit-hep](https://scikit-hep.org/)
 - [Neuroimaging](https://nipy.org/nibabel/) - nipy uses `ndarray` as the fundamental structure for the entire stack
   * fMRI visualization example from [section 3.4](https://www.frontiersin.org/articles/10.3389/fninf.2014.00014/full#h4)
     is a nice, brief example

## Neuroimaging Analysis

Like much of the scientific python ecosystem, the [nipy organization](https://nipy.org/) relies on `np.ndarray` as the fundamental structure for neuroimaging data

The following example is adapted from [Machine learning for neuroimaging with scikit learn](https://www.frontiersin.org/articles/10.3389/fninf.2014.00014/full). The dataset used comes from the [nilearn data](https://www.nitrc.org/frs/?group_id=728).

<font color=red>**Add example of loading full Nifti image to show 4D structure of data?**</font>

In [ ]:
import nibabel   # package for loading/saving neuroimaging data
bg_img = nibabel.load('data/bg.nii.gz')
bg = bg_img.get_fdata()
type(bg)

In [ ]:
# Create activation map by thresholding the data
act_thresh = 6000
act = bg.copy()
# Set "unactivated" voxels to NaN for visualization
act[act <= act_thresh] = np.nan

In [ ]:
# Set "unactivated" voxels to NaN for visualization

imshow_opts = {
    "origin" : "lower",
    "interpolation" : "nearest"
}

# Axial slice of activation map overlay
plt.imshow(bg[...,10].T, cmap="gray");             # Background
plt.imshow(act[...,10].T, cmap="plasma");   # Activation map
plt.axis('off')
plt.show()

## Detecting gravitational wave signature of black hole and neutron star mergers

[PyCBC](https://pycbc.org/) is the toolkit used to analyze data from gravitational wave observatories like [LIGO](https://www.ligo.caltech.edu/) and [Virgo](http://www.virgo-gw.eu/).

The [PyCBC tutorials](https://github.com/gwastro/PyCBC-Tutorials) have some really cool examples - let's recreate the "chirp" from [first ever direct detection of gravitational waves](https://en.wikipedia.org/wiki/First_observation_of_gravitational_waves) that resulted from two black holes merging. For more info, see [the second PyCBC tutorial](https://colab.research.google.com/github/gwastro/pycbc-tutorials/blob/master/tutorial/2_VisualizationSignalProcessing.ipynb).

In [ ]:
import pycbc

merger_data = pycbc.catalog.Merger('GW150914')
# Though the catalog includes data from multiple observatories,
# let's focus on just one
ligo_data = merger_data.strain('L1')
type(ligo_data)

In [ ]:
# PyCBC has it's own (quite extensive) API that uses
# numpy & scipy under the hood
print(type(ligo_data._data))
pycbc.types.aligned.ArrayWithAligned.__bases__

To re-create the "chirp" we have to do some analysis on the raw data. `pycbc` relies on tools in `scipy.fft` and `scipy.signal` to implement the frequency analysis.

In [ ]:
# Flatten frequency 
res = ligo_data.whiten(4, 4)

In [ ]:
time_of_merger = merger_data.time

# Look 1-seconds worth of data around the merger time
roi = res.time_slice(time_of_merger - 0.3, time_of_merger + 0.3)

# Plot the spectrogram
times, freqs, power = roi.qtransform(
    delta_t=0.001,
    logfsteps=100,
    qrange=(8, 8),
    frange=(30, 512),
)

fig, ax = plt.subplots(figsize=(6,3))
ax.pcolormesh(times, freqs, power**0.5)
ax.set_yscale('log')

# Scope of NumPy

NumPy currently targets computation involving:

 * in-memory, homogenously-typed array data
 * cpu-based
 
Important guiding principles:
 - **Stability**: Foundational component of the scientific python ecosystem for going-on 15 years
 - **Interoperability**: A *de facto* standard for array APIs in python

# Adapting to community needs

 - In the early days, many new NumPy users were converts from matlab
   * See the [NumPy for Matlab users](https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html) article in the docs
   
 - Now: The scientific ecosystem is incredibly feature-rich and powerful: attracts many new users
   * Users interested in specific applications (machine learning, image processing, geoscience, bioinformatics, etc.) end up interacting with NumPy indirectly
   * Focus resources on supporting stable, performant base for dependent libraries
     * Why scope is important: what goes in NumPy itself vs. dependent packages?
     * Balance between performance and maintainability
       > Optimization is the altar where maintainability is sacrificed
       >
       > L. Ramalho, *Fluent Python*

## Google Trends

In [ ]:
timeseries_dtype = np.dtype([
    ('date', 'datetime64[M]'),
    ('relpop', float)
])

parse_kwargs = {
    "skiprows" : 3,
    "delimiter" : ",",
    "dtype" : timeseries_dtype
}

gtrends_search_terms = ("NumPy", "Data Science", "Matlab")
fnames = [name.lower().replace(' ', '') for name in gtrends_search_terms]

data = {
    fname : np.loadtxt("data/{}.csv".format(fname), **parse_kwargs) for fname in fnames
}

fig, ax = plt.subplots()
for name, vals in data.items():
    plt.plot(vals['date'], vals['relpop'], label=name)
ax.set_title('Google Trends (US): 2004 - Present')
ax.set_ylabel('Relative Popularity of Search Term [arb]')
ax.legend();

In [ ]:
def smooth(s, kernsize=21):
    s_padded = np.r_[s[kernsize-1:0:-1], s, s[-2:-kernsize-1:-1]]
    kern = np.hamming(kernsize)
    res_padded = np.convolve(kern/kern.sum(), s_padded, mode='valid')
    # De-pad and renormalize
    return res_padded[kernsize//2:-kernsize//2+1] / res_padded.max()

fig, ax = plt.subplots()
for name, vals in data.items():
    plt.plot(vals['date'], smooth(vals['relpop']), label=name)
ax.set_title('Google Trends (US): 2004 - Present')
ax.set_ylabel('Relative Popularity of Search Term [arb]')
ax.legend();

# How is NumPy Developed

 - Collaboratively (caveat here about the bus factor)

Commitment to stability means proposed changes must go through extensive design and review:
 - [NEPs](https://numpy.org/neps/) - analogous to PEPs, specific to NumPy
 - Steering council for high-level direction and coordination with [NumFOCUS](https://numfocus.org/)

# Case-Study: `np.random`

 - Overhaul of `np.random` landed in version 1.17
 
   * Improve *performance* and *flexibility* without sacrificing stability

In [ ]:
# Generate 1,000,000 random numbers the old way
old_rands = np.random.random(int(1e6))
print("Uniform random numbers from legacy np.random.random:\n  {}".format(old_rands))

# ... and the new way
from numpy.random import PCG64, Generator
rg = Generator(PCG64())
new_rands = rg.random(int(1e6))
print("Uniform random numbers with new tools:\n  {}".format(new_rands))

## Compatibility

Before version 1.17, `numpy.random` relied on `RandomState` to configure and produce random numbers.

There are many, many LOC (both in test suites and in production) that depend on the original `numpy.random`, so both the *interface* and the *implementation* must remain unchanged
 * <font color="green">**Upside:**</font> output of `np.random` remains "stable"
 * <font color="orange">**Downside:**</font> users know about new interface to access improvements

In [ ]:
# Choose a seed for generator
seed = 1817

# Random numbers generated by np.random in v1.15
rands_from_v1_15 = np.load('data/npy_v1.15_random_seed1817_1000samples.npy')
# Generate random numbers with legacy interface
np.random.seed(seed)
legacy_rands = np.random.random(1000)

print("Arrays equivalent: ", np.allclose(rands_from_v1_15, legacy_rands))

In [ ]:
# It is possible (though clunky) to replicate legacy behavior with new interface
seed = 1817

from numpy.random import MT19937, RandomState
# Set random state with legacy seeding
rs = RandomState(seed)
mt = MT19937()
mt.state = rs.get_state()

# New interface for generation
mt_gen = Generator(mt)
mt_rands = mt_gen.random(1000)
print("Arrays equivalent: ", np.allclose(legacy_rands, mt_rands))

## Performance

`Generator` includes improved methods for drawing samples from distributions.

In [ ]:
#NOTE: PCG64 is the new default bit_generator, so the following is equivalent to Generator(PCG64())
from numpy.random import default_rng
rg = default_rng()
num_samples = int(1e5)

In [ ]:
print("Standard Normal:")
%timeit np.random.standard_normal(num_samples)
%timeit rg.standard_normal(num_samples)

In [ ]:
print("Standard Exponential:")
%timeit np.random.standard_exponential(num_samples)
%timeit rg.standard_exponential(num_samples)

In [ ]:
print("Standard Gamma:")
shape_param = 3.0
%timeit np.random.standard_gamma(shape_param, num_samples)
%timeit rg.standard_gamma(shape_param, num_samples)

In [ ]:
# parallel example too?

# What's next for NumPy?

<font color=red>**TODO: UPDATE IMAGE**</font>

![Numpy Roadmap](./images/numpy_roadmap_graphic.png)

Image from [this PyData Amsterdam 2019 presentation](https://www.slideshare.net/RalfGommers/the-evolution-of-array-computing-in-python/14) by [Ralf Gommers](https://github.com/rgommers)

## Slide(s) on interop

 - Ralf's material from [here](https://www.slideshare.net/RalfGommers/pydata-nyc-whatsnew-numpyscipy-2019?next_slideshow=1)

## Slide(s) on dtype work

## Slide(s) on SIMD work

## Slides on indexing?

## Slide(s) on type annotations

# And beyond?

NumPy 2.0? 

# Getting involved

Great opportunity to work on a project that is depended on by tens of millions of users (and counting)

What can you do:

 1. Contribute
 
   - [GitHub Issues](https://github.com/numpy/numpy/issues) and [open PRs](https://github.com/numpy/numpy/pulls) are a great entry point
     * If you want to get your hands dirty immediately, try starting with the [good first issue](https://github.com/numpy/numpy/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22) label
     * For challenges with a greater scope, try the [Enhancement](https://github.com/numpy/numpy/labels/01%20-%20Enhancement) or [Wish List](https://github.com/numpy/numpy/labels/23%20-%20Wish%20List) labels
   - Check out the discussion revolving around current and future [NEPs](https://numpy.org/neps/)
   

 2. Participate in the conversation
 
  - [Numpy discussion mailing list](https://www.scipy.org/scipylib/mailing-lists.html)
  - Numpy community meetings (links and cadence here)
  - slack channel